We want to compare in computational terms Tridiagonal matrixes and Sparse matrixes with the "\" solver and other solving algorithms

# 1.1 Functions and Pakages:
Now we go to comparing the two solver (\ and "LinearSolve pakage") with both sparse and tridiagonal matrix.
We starting by making a function for it :  

In [7]:
using SparseArrays
using Plots
using LinearSolve
using BenchmarkTools
using LinearAlgebra

function solver(Mat,b)
    if(size(Mat)[1]==size(Mat)[2])
        if(size(Mat)[1]!=size(b)[1])
            error("Wrong dimensions")
        end
        u = Mat \ b
        return u::Vector{Float64}
    else
        error("It's not a square matrix")
    end
end

function solv(Mat,b::Vector{Float64}) 
    if(size(Mat)[1]==size(Mat)[2])
        if(size(Mat)[1]!=size(b)[1])
            error("Wrong dimensions")
        end
        prob = LinearProblem(Mat, b)
        u = solve(prob, KrylovJL_CG())
        return u
    else
        error("It's not a square matrix")
    end
end


solv (generic function with 1 method)

# 1.2: Do the test with \ solver:
 In this first snippet of code we compared the "\" solver with a randomized 10000x10000 matrix with the 90% of zero-elements stored as a sparse one "A" and as a full one "B". We can see that in the first case it takes 5 times more time to find the u vector, and several more bytes of memory used. 

In [8]:
n = 10000;
density = 0.1;
f = rand(n);
A = sprand(n, n, density);
display(@btime solver(A,f))
B = Matrix(A);
display(@btime solver(B,f));

10000-element Vector{Float64}:
  28.922624218729478
 -22.454140067209764
  13.13545141236079
  -9.015338424316582
  17.60601560338772
 -15.835226828434118
 -18.740705813960236
  16.085916665458992
  16.59444619709569
 -38.6117614690437
   ⋮
  24.2999851112344
 -14.262701568317308
 -11.713622984363255
  13.8505774312978
  -8.69130363066101
  15.283862920875114
 -10.249345215160233
   1.6594990431631256
  -6.026067649789386

  13.633 s (85 allocations: 3.81 GiB)


10000-element Vector{Float64}:
  28.92262422378809
 -22.454140070255406
  13.13545141398023
  -9.015338425198898
  17.606015604244988
 -15.835226830435765
 -18.740705817120265
  16.085916667467355
  16.59444619917311
 -38.61176147460617
   ⋮
  24.299985114108104
 -14.262701570203781
 -11.713622985826369
  13.850577433056825
  -8.69130363176044
  15.28386292265571
 -10.249345216301704
   1.659499043123873
  -6.02606765051643

  4.015 s (6 allocations: 763.09 MiB)


# 1.3: With random matrix: 
In this second snippet of code we compared the "\" solver with a randomized 10000x10000 tridiagonal matrix stored as a sparse one "C" and as a full one "D". Now we see that storing the matrix as sparse led us to a way faster resolution of the problem using the base solver.

In [10]:
n=10000
f = rand(n);

#generating the low, high and main diagonal of a tridiagonal matrix
hd = rand(n-1);
ld = rand(n-1);
d = rand(n)

C = Tridiagonal(ld,d,hd)
D = Matrix(C)
display(@btime solver(C,f))
display(@btime solver(D,f))

10000-element Vector{Float64}:
   3.930000176734958
   0.5432587866937981
  -1.7762906303950157
   1.0413557220233771
   1.217974892489639
  -1.3252071891906876
   0.6566868297063173
   1.3991911975433198
  -2.0354919702341587
   4.2018597539726334
   ⋮
   9.21995892924822
   0.2275527000000487
 -18.05210933265012
  19.668151837843656
  -9.657949769412788
   2.3497877938772382
   4.936125560170783
 -13.123670936089871
  14.112697091495184

  162.000 μs (14 allocations: 469.12 KiB)


10000-element Vector{Float64}:
   3.930000176734995
   0.543258786693795
  -1.776290630395033
   1.0413557220233824
   1.2179748924896439
  -1.3252071891907027
   0.6566868297063219
   1.399191197543335
  -2.0354919702342005
   4.201859753972659
   ⋮
   9.219958929248218
   0.2275527000000487
 -18.05210933265012
  19.668151837843656
  -9.657949769412788
   2.3497877938772382
   4.936125560170783
 -13.123670936089871
  14.112697091495184

  3.971 s (6 allocations: 763.09 MiB)


So we can concluse that the best way to store the matrix is tridiagonal, in this way with the base solver we use less computational time.

# 2.1: Do the test with Linearsolver:
In order to be our comparasion more justified we do the tests with the Same matrix A,B,C and D :

# 2.2:  Sparse/Tridiagonal  Matrix:

In [9]:
display(@btime u=solv(A,f))
display(@btime u=solv(B,f))

u: 10000-element Vector{Float64}:
  3.950986093537063e14
 -4.699138595088857e14
 -1.2215685492947175e15
  3.82990716265518e13
  8.727495961437848e14
 -5.6226610332671e14
 -2.3407572981451625e14
  6.771649439930036e14
  7.350212057068736e14
 -1.5286629147377085e15
  ⋮
 -7.644981425002366e14
 -1.2477082666805738e15
 -1.7080997050129928e15
  1.4499251534559142e15
 -1.663521285606508e15
 -5.174267526055381e14
 -1.236544395588974e15
 -6.415012248925021e14
 -1.580275486161275e15

  141.409 ms (35 allocations: 392.86 KiB)

u: 10000-element Vector{Float64}:
  3.4546215805780144e14
 -4.108783280021132e14
 -1.0681022338833322e15
  3.3487538610579367e13
  7.631055938695852e14
 -4.9162830964080844e14
 -2.046686696846477e14
  5.920923470107369e14
  6.426800953744296e14
 -1.3366161686371548e15
  ⋮
 -6.684538287070416e14
 -1.0909580044817425e15
 -1.4935102182134478e15
  1.2677702747537445e15
 -1.454532209669806e15
 -4.5242215072421394e14
 -1.0811966565339671e15
 -5.6090908016760506e14
 -1.3817446248877695e15


  249.758 ms (33 allocations: 392.70 KiB)


# 2.2:  Sparse/Tridiagonal  Matrix with random matrix:

In [11]:
display(@btime solver(C,f))
display(@btime solver(D,f))

10000-element Vector{Float64}:
   3.930000176734958
   0.5432587866937981
  -1.7762906303950157
   1.0413557220233771
   1.217974892489639
  -1.3252071891906876
   0.6566868297063173
   1.3991911975433198
  -2.0354919702341587
   4.2018597539726334
   ⋮
   9.21995892924822
   0.2275527000000487
 -18.05210933265012
  19.668151837843656
  -9.657949769412788
   2.3497877938772382
   4.936125560170783
 -13.123670936089871
  14.112697091495184

  164.100 μs (14 allocations: 469.12 KiB)


10000-element Vector{Float64}:
   3.930000176734995
   0.543258786693795
  -1.776290630395033
   1.0413557220233824
   1.2179748924896439
  -1.3252071891907027
   0.6566868297063219
   1.399191197543335
  -2.0354919702342005
   4.201859753972659
   ⋮
   9.219958929248218
   0.2275527000000487
 -18.05210933265012
  19.668151837843656
  -9.657949769412788
   2.3497877938772382
   4.936125560170783
 -13.123670936089871
  14.112697091495184

  4.033 s (6 allocations: 763.09 MiB)
